Data Ingestion from S3 to Snowflake 

In [ ]:
USE ROLE ACCOUNTADMIN;
USE DATABASE SNOWFLAKE_LEARNING_DB;
USE WAREHOUSE SNOWFLAKE_LEARNING_WH; 
USE SCHEMA AKUMAR2104_LOAD_SAMPLE_DATA_FROM_S3; 



In [ ]:

CREATE OR REPLACE STAGE AWSS3_STAGE 
URL = 's3://sfquickstarts/tastybytes/' ;

CREATE OR REPLACE FILE FORMAT MENU_FILE_FF
TYPE = CSV; 
---IGNORE_UTF8_ERRORS = TRUE;




In [ ]:
CREATE OR REPLACE TABLE MENU
(
    menu_id NUMBER(19,0),
    menu_type_id NUMBER(38,0),
    menu_type VARCHAR(16777216),
    truck_brand_name VARCHAR(16777216),
    menu_item_id NUMBER(38,0),
    menu_item_name VARCHAR(16777216),
    item_category VARCHAR(16777216),
    item_subcategory VARCHAR(16777216),
    cost_of_goods_usd NUMBER(38,4),
    sale_price_usd NUMBER(38,4),
    menu_item_health_metrics_obj VARIANT
);

select * from menu

In [ ]:
copy into menu 
from  @AWSS3_STAGE/raw_pos/menu/ 
file_format = (format_name = 'MENU_FILE_FF');


In [ ]:
SELECT
   menu_item_name
FROM menu
WHERE truck_brand_name = 'Freezing Point';

In [ ]:

SELECT
    m.menu_item_name,
    obj.value:"ingredients"::ARRAY AS ingredients
FROM menu m,
    LATERAL FLATTEN (input => m.menu_item_health_metrics_obj:menu_item_health_metrics) obj
WHERE TRUE
AND truck_brand_name = 'Freezing Point'
AND menu_item_name = 'Mango Sticky Rice';